# Functional abstraction

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The topics discussed are:
- we now go back to our meta-language, so let us forget about Python for a moment
- programming as giving (the right) names
- variables as names of expressions
- names as a problem specific taxonomy
    - fundamental elements of the problem also become fundamental elements of the solution
- what about giving names to code?
    - hollow square example: the first and last line are duplicated
    - the issue with duplication is reasoning overhead, but even worse wrong level of abstraction: we are not mentioning the right concepts
- in order to give a name to code, we need to be able to bind names to code
    - the state must be extended with a new sort of value: the "code value"
    - code values are known as _lambda expressions_
    - for our first attempt we will define a lambda as `<() => L>`, where `L` is a statement called _body_
        - notice the usual quotes around code, since we are treating code as an object passed around and inspected, just like in our $\text{eval}$ function
    - our state will be able to contain code, in the format:
    
        $$\{ x := 0; \text{incr} := \texttt{<() => x := x + 1>} \}$$
        
    - we can then use code from the state by _invoking_ it
    - invocation has a simple syntax: `V()`, where `V` is the name of a variable bound to a lambda
    - the semantics of invocation are quite trivial: we simply replace the invocation statement with the code from the lambda:
        $\text{eval}(<V()>, S) \rightarrow <L>, S$ where $S[V] \rightarrow <() \Rightarrow L>$
    - the combination of code in the state and the variable(s) to which the code is bound are collectively called _function_
- let us see an example in depth, by following the steps of its evaluation:

```
x := 0
y := 0
incr_x := <() => x := x + 1>
incr_y := <() => y := y + x>
incr_x()
incr_x()
incr_y()
```

- notice that we can reuse the code for incrementing `x` without having to repeat it, and that `incr_x()` is less of a mouthful than `x := x + 1`
    - on a larger scale, this makes it harder to make mistakes
- let us see a more complex example taken from our square-drawing from previous lectures
- we would like to be able to specify that drawing a square requires drawing a line multiple times
- this means we could redefine our sample as:

```
n := int(input())
s := ""
draw_line := <
  () =>
    x := n
    while x > 0 do
      s := s + "*"
      x := x - 1
    s := s + "\n"
  >
y := n
while y > 0
  draw_line()
  y := y - 1
```

- this process of identifying interesting parts of our code and giving them a name is also known as _abstraction_
    - abstraction literally means "removal"
    - in our case, we remove details
    - saying `draw_line()` is far more direct and concise than its whole code
- moreover, we can keep going in our process of abstraction until there is little left to abstract away:

```
n := int(input())
s := ""
draw_line := <
  () =>
    x := n
    while x > 0 do
      s := s + "*"
      x := x - 1
    s := s + "\n"
  >
draw_N_lines := <
  () =>
    y := n
    while y > 0
      draw_line()
      y := y - 1
  >

draw_N_lines()
```

- in this new formulation we are simply saying that drawing a square is just the same as "drawing N lines"
    - "drawing N lines" does simply this: it "draws a line" `n` times
    - "drawing a line" simply draws `n` asterisks, followed by a `"\n"` symbol
    - our program is now getting more readable, by hiding details behind (the names of) our functions
    - if we want to know what hides behind the name, we can simply go and check the function definition, that is the place in the program where the function variable was bound for the first time

- sometimes, defining a function requires a little more context
- for example, suppose we wanted to extend the definition of `incr_x` by also specifying by how much we want `x` to be incremented
    - this is trickier, because now invoking the function will also require defining this extra information
    - we call this extra information _parameter_ of the function, and we say the function is "parameterized by it"
    - we define a parameterized function as follows: `<(Pars) => L>`, where `L` is a statement and `Pars` is a series of variable names separated by comma
    - for example, we could now use the following assignment in a program: `incr_x_by := <(k) => x := x + k>`
    - the function `incr_x_by` will now increment `x` by `k`, which is a parameter of the function
    - invocation supplies _arguments_ to the function
        - the arguments are assigned to the parameters right before the function body is injected in the main program
    - this leads us to the following semantics:
    
        $\text{eval}(<V(a_1, a_2, \dots)>, S) \rightarrow <p_1 := a_1; p_2 := a_2; \dots; L>, S$ where $S[V] \rightarrow <(p_1, p_2, \dots) \Rightarrow L>$ and $a_1, a_2, \dots$ are all constant values.
        
        Note that we cannot directly assign the arguments $a_1, a_2, ...$ to the parameters $p_1, p_2, ...$ inside the state, because the arguments could be expressions (for example, 3 + 1) instead of atomic values, so we need to execute the code to evaluate it. This means that, before the rule above can become active, we need to check if some of the arguments are not (yet) constant, in a left-to-right fashion:
        
        $\text{eval}(<V(\dots, E_i, \dots)>, S) \rightarrow <\dots, E_i', \dots; L>, S$ where $\text{eval}(<E_i>,S) \rightarrow <E_i'>,S'$ and $E_i$ is the first argument from the left which is not a constant.
        
- as an example, consider the evaluation steps of the following program:

```
incr_x_by_k := <(k) => x := x + k>
x := 0
incr_x_by_k(1)
incr_x_by_k(2)
incr_x_by_k(x)
```

- notice that at each invocation we reassign `k` to the value between the brackets
- we can also have multiple parameters in the function definition, and therefore multiple arguments when invoking the function
    - show a detailed expansion of the code below

```
mul_incr := <(c,k) => x := x * c + k>
x := 1
mul_incr(5, 3)
mul_incr(2, 1)
```

